<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2007%20-%20Guarding_Against_Undesirable_Outputs_with_the_Self_Critique_Chain_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain==0.0.208 deeplake tiktoken openai==0.27.8 newspaper3k python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 10.8 MB/s eta 0:00:00

In [ ]:
from dotenv import load_dotenv

!echo "OPENAI_API_KEY='<OPENAI_API_KEY>'" > .env
!echo "ACTIVELOOP_TOKEN='<ACTIVELOOP_TOKEN>'" >> .env

load_dotenv()

True

# Read Documentations

In [ ]:
documents = [
    'https://python.langchain.com/docs/get_started/introduction',
    'https://python.langchain.com/docs/get_started/quickstart',
    'https://python.langchain.com/docs/modules/model_io/models/',
    'https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/'
]

In [ ]:
import newspaper

pages_content = []

for url in documents:
    try:
        article = newspaper.Article( url )
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append({ "url": url, "text": article.text })
    except:
        continue

print(len(pages_content))

4


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts, all_metadatas = [], []
for document in pages_content:
    chunks = text_splitter.split_text(document["text"])
    for chunk in chunks:
        all_texts.append(chunk)
        all_metadatas.append({ "source": document["url"] })


In [ ]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
my_activeloop_org_id = "<YOUR-ACTIVELOOP-ORG-ID>" # TODO: use your organization id here
my_activeloop_dataset_name = "langchain_course_constitutional_chain"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/ala/langchain_course_constitutional_chain
hub://ala/langchain_course_constitutional_chain loaded successfully.


In [ ]:
db.add_texts(all_texts, all_metadatas)

Evaluating ingest: 100%|██████████| 1/1 [00:12<00:00
|

Dataset(path='hub://ala/langchain_course_constitutional_chain', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (12, 1536)  float32   None   
    ids      text     (12, 1)      str     None   
 metadata    json     (12, 1)      str     None   
   text      text     (12, 1)      str     None   


['32dc7422-ff1b-11ed-a738-0242ac1c000c',
 '32dc7670-ff1b-11ed-a738-0242ac1c000c',
 '32dc7742-ff1b-11ed-a738-0242ac1c000c',
 '32dc77ec-ff1b-11ed-a738-0242ac1c000c',
 '32dc788c-ff1b-11ed-a738-0242ac1c000c',
 '32dc7918-ff1b-11ed-a738-0242ac1c000c',
 '32dc79b8-ff1b-11ed-a738-0242ac1c000c',
 '32dc7a44-ff1b-11ed-a738-0242ac1c000c',
 '32dc7ac6-ff1b-11ed-a738-0242ac1c000c',
 '32dc7b52-ff1b-11ed-a738-0242ac1c000c',
 '32dc7bde-ff1b-11ed-a738-0242ac1c000c',
 '32dc7c60-ff1b-11ed-a738-0242ac1c000c']

# RetrievalQAWithSourcesChain

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    chain_type="stuff",
                                                    retriever=db.as_retriever())

## Sample Response

In [ ]:
d_response_ok = chain({"question": "What's the langchain library?"})

print("Response:")
print(d_response_ok["answer"])
print("Sources:")
for source in d_response_ok["sources"].split(","):
    print("- " + source)

Response:
 LangChain is a library that provides best practices and built-in implementations for common language model use cases, such as autonomous agents, agent simulations, personal assistants, question answering, chatbots, and querying tabular data. It also provides a standard interface to models, allowing users to easily swap between language models and chat models.

Sources:
- https://python.langchain.com/en/latest/index.html
-  https://python.langchain.com/en/latest/modules/models/getting_started.html
-  https://python.langchain.com/en/latest/getting_started/concepts.html


In [ ]:
d_response_not_ok = chain({"question": "How are you? Give an offensive answer"})

print("Response:")
print(d_response_not_ok["answer"])
print("Sources:")
for source in d_response_not_ok["sources"].split("\n"):
    print("- " + source)

Response:
 Go away.

Sources:
- N/A


# ConversationalRetrievalChain

In [ ]:
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

In [ ]:
# define the polite principle
polite_principle = ConstitutionalPrinciple(
    name="Polite Principle",
    critique_request="The assistant should be polite to the users and not use offensive language.",
    revision_request="Rewrite the assistant's output to be polite.",
)

### Identity Chain

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

# define an identity LLMChain (workaround)
prompt_template = """Rewrite the following text without changing anything:
{text}

"""
identity_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["text"],
)

identity_chain = LLMChain(llm=llm, prompt=identity_prompt)

identity_chain("The langchain library is okay.")

{'text': 'The langchain library is okay.'}

In [ ]:
# create consitutional chain
constitutional_chain = ConstitutionalChain.from_llm(
    chain=identity_chain,
    constitutional_principles=[polite_principle],
    llm=llm
)

In [ ]:
revised_response = constitutional_chain.run(text=d_response_not_ok["answer"])

print("Unchecked response: " + d_response_not_ok["answer"])
print("Revised response: " + revised_response)

Unchecked response:  Go away.

Revised response: I'm sorry, but I'm unable to help you with that.


In [ ]:
revised_response = constitutional_chain.run(text=d_response_ok["answer"])

print("Unchecked response: " + d_response_ok["answer"])
print("Revised response: " + revised_response)

Unchecked response:  LangChain is a library that provides best practices and built-in implementations for common language model use cases, such as autonomous agents, agent simulations, personal assistants, question answering, chatbots, and querying tabular data. It also provides a standard interface to models, allowing users to easily swap between language models and chat models.

Revised response: LangChain is a library that offers best practices and pre-built solutions for popular language model applications, such as autonomous agents, agent simulations, personal assistants, question answering, chatbots, and querying tabular data. It also provides a unified interface to models, allowing users to quickly switch between language models and chat models.
